# Practica 2B

## Ascensores

    
El objetivo de esta práctica es aplicar las las técnicas de representación de problemas y los algoritmos de búsqueda a un problema real.

Se dispone de un edificio de 13 plantas (numeradas del 0 al 12). El edificio está organizado en tres bloques de 5 pisos cada uno y comparten el piso de comunicación. Es decir, los bloques 1 y 2 comparten la planta 4, y los bloques 2 y 3 comparten la planta 8. El bloque 1 va de la planta 0 a la planta 4; el bloque 2 va de la planta 4 a la planta 8; el bloque 3 va de la planta 8 a la planta 12.
En el edificio hay un ascensor rápido que sólo para en las plantas pares (0,2,4,6,8,10,12). Además, cada bloque dispone de uno o dos ascensores lentos, tal y como se muestra en la figura, que solo se pueden mover entre las plantas del bloque correspondiente. El ascensor rápido tiene una capacidad para tres personas, y los ascensores lentos tienen capacidad para dos personas. En el caso de usar un ascensor lento para cambiar de bloque es necesario cambiar de ascensor.
Ejemplo: para ir de la planta 1ª a la 12ª hay que coger un ascensor lento desde 1ª hasta 4ª planta, otro de la 4ª a la 8ª y por último de la 8ª a la 12ª.

Se pide definir la representación del problema para su resolución usando el paradigma del espacio de estados.
Tenemos que resolver una situación inicial en la que hay 5 pasajeros que están situados en las siguientes plantas:
* Javier (Pasajero 0) está en la planta 2
* Alvaro (Pasajero 1) está en la planta 4
* Ana (Pasajero 2) está en la planta 1
* Irene (Pasajero 3) está en la planta 8
* Belen (Pasajero 4) está en la planta 1

El objetivo es llevar a los pasajeros a las siguientes plantas:
* Javier quiere ir a la planta 3
* Álvaro quiere ir a la planta 11
* Ana quiere ir a la planta 12
* Irene quiere ir a la planta 1
* Belen quiere ir a planta 9

a) Discute alternativas para la función de coste y heurísticas para el problema, así como sus propiedades de admisibilidad y consistencia.
Se puede suponer que el ascensor rápido tarda la mitad que los lentos y que todos los lentos tardan igual. Aunque el objetivo final es que todas las personas lleguen a su destino en el menor tiempo total también se pueden considerar otros aspectos como el tiempo de espera de las personas en las plantas o el número de paradas. En el problema original los ascensores se pueden mover a la vez por el edificio, pero si crees que es necesario hacer alguna simplificación del problema deberás justificar la respuesta adecuadamente.

b) Especificar otras instancias de problema cambiando la situación inicial y final y cómo afectan a la resolución del problema.

c) Explicar si la representación elegida permite modificar los siguientes datos:
- Cambiar plantas de origen y destino de los pasajeros
- Tener en cuenta el tiempo de subir y bajar pasajeros para cambiar de ascensor.
- Incluir nuevos pasajeros
- Cambiar la capacidad de los ascensores
- Añadir nuevos ascensores, por ejemplo, un ascensor rápido que se mueva entre las plantas impares.

d) Incluir análisis de resultados (eficiencia, optimalidad) de los algoritmos y heurísticas elegidos. Se debe incluir una comparativa entre las mejores opciones estudiadas.

# Clase y función para analizar los resultados.

In [3]:
from search import *

class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)
    
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return self.problem.path_cost(c, state1, action, state2)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable 
        Para el problema de los ascensores siempre hay una solucion"""

        return True

In [4]:
def resuelve_problema(problem, algoritmo, h=None):
    problema=Problema_con_Analizados(problem)
    sol = []
    import time
    start = time.time()
    if h:
        sol= algoritmo(problema,h).solution()
    else: 
        sol= algoritmo(problema).solution()
    end = time.time()
    
    print("Solución:")
    i = 1
    for s in sol:
        solucion = s
        if "ASCENSORES" in s:
            solucion = ",".join(s.split()[2:])
            solucion = " ".join(solucion.split("_"))
            solucion = " ".join(solucion.split("."))
            solucion = ", ".join(solucion.split(","))
        text = "\tPaso {NUM}: {SOL}"
        print(text.format(NUM=i,SOL=solucion.capitalize()))
        i += 1

    print("\nTiempo del algoritmo: " + str( '%.4f'  % (end - start)) + " segundos")
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    print("Longitud de la solución: {0}.".format(len(sol)))
    print("Nodos analizados: {0}".format(problema.analizados))

## SOLUCIÓN CON UN SOLO PASAJERO Y UN ASCENSOR

In [4]:
from search import *

class ProblemaAscensoresV1(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self._limiteAscLento = 2


    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        accs = list()

        pisoPas1 = s[0]
        cantPasAsc1 = s[1]
        pisoAsc1 = s[2]
        
        
        if (pisoAsc1 > 0):
            accs.append("Bajar")
        
        if (pisoAsc1 < 12):
            accs.append("Subir")
            
        if (pisoAsc1 == pisoPas1):
            accs.append("Subir pasajero")
            
        if (cantPasAsc1 > 0):
            accs.append("Bajar pasajero")
        
        return accs

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        
        pisoPas1 = s[0]
        cantPasAsc1 = s[1]
        pisoAsc1 = s[2]
        
        if (a == "Subir"):
            if (cantPasAsc1 > 0):
                return (pisoPas1 + 1, cantPasAsc1, pisoAsc1 + 1)
            else:
                return (pisoPas1, cantPasAsc1, pisoAsc1 + 1)
            
        if (a == "Bajar"):
            if (cantPasAsc1 > 0):
                return (pisoPas1 - 1, cantPasAsc1, pisoAsc1 - 1)
            else:
                return (pisoPas1, cantPasAsc1, pisoAsc1 - 1)
            
        if (a == "Subir pasajero"):
            return (pisoPas1, cantPasAsc1 + 1, pisoAsc1)
        
        if (a == "Bajar pasajero"):
            return (pisoPas1, cantPasAsc1 - 1, pisoAsc1)
    
    def goal_test(self, s):
        pisoPas1 = s[0]
        objPas1 = self.goal[0]        
        pasAsc1 = s[1]
        
        ascensoresVacios = (pasAsc1 == 0)
        
        pasajerosEnSuPiso = (pisoPas1 == objPas1)
        
        return pasajerosEnSuPiso and ascensoresVacios
    
    def h1(self, node):        
        pisosPasajeros = node.state[0]
        pisosObjetivos = self.goal[0]
                
        return abs(pisosPasajeros - pisosObjetivos)
        

### Representacion:

(pisPas1, #pasAsc1, pisoA1) -> (2,0,0) -> (3,0,0)

In [5]:
estado_inicial_e1 = (2,0,0)
objetivo_e1 = (3,0,0)
problema_1 = ProblemaAscensoresV1(estado_inicial_e1, objetivo_e1)
resuelve_problema(problema_1, astar_search, problema_1.h1)

Solución:
	Paso 1: Subir
	Paso 2: Subir
	Paso 3: Subir pasajero
	Paso 4: Subir
	Paso 5: Bajar pasajero

Tiempo del algoritmo: 0.0002 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 5.
Nodos analizados: 9


In [6]:
estado_inicial_e1 = (1,0,0)
objetivo_e1 = (12,0,0)
problema_1 = ProblemaAscensoresV1(estado_inicial_e1, objetivo_e1)
resuelve_problema(problema_1, astar_search, problema_1.h1)

Solución:
	Paso 1: Subir
	Paso 2: Subir pasajero
	Paso 3: Subir
	Paso 4: Subir
	Paso 5: Subir
	Paso 6: Subir
	Paso 7: Subir
	Paso 8: Subir
	Paso 9: Subir
	Paso 10: Subir
	Paso 11: Subir
	Paso 12: Subir
	Paso 13: Subir
	Paso 14: Bajar pasajero

Tiempo del algoritmo: 0.0022 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 14.
Nodos analizados: 38


## SOLUCIÓN CON AGRUPACIÓN DE PASAJEROS Y UN SOLO ASCENSOR

In [19]:
from search import *

class ProblemaAscensoresV2(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self._limiteAscLento = 2


    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        accs = list()

        pasAsc1 = s[1]
        cantPasAsc1 = sum(pasAsc1)
        pisoAsc1 = s[2]
        pisosPasajeros = s[0]
        
        
        if (pisoAsc1 > 0):
            accs.append("Bajar")
        
        if (pisoAsc1 < 12):
            accs.append("Subir")
            
        i = 1
        for pisoPas in pisosPasajeros:
            if ((pisoPas == pisoAsc1) and (cantPasAsc1 < self._limiteAscLento)):
                accs.append("Subir pasajero " + str(i))
            i += 1
        
        i = 1
        for pas in pasAsc1:
            if (pas == 1):
                accs.append("Bajar pasajero " + str(i))
            i += 1
        
        return accs
    
    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        
        pasAsc1 = s[1]
        pisoAsc1 = s[2]
        pisosPasajeros = s[0]
        
        if (a == "Subir"):
            if (1 in pasAsc1):
                dentroAsc1 = []
                
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                    
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc1:
                    nuevosPisos[index] += 1
                
                return (tuple(nuevosPisos), pasAsc1, pisoAsc1 + 1)
            
            return (pisosPasajeros, pasAsc1, pisoAsc1 + 1)
        
        if (a == "Bajar"):
            if (1 in pasAsc1):
                dentroAsc1 = []
                
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc1:
                    nuevosPisos[index] -= 1
                    
                
                return (tuple(nuevosPisos), pasAsc1, pisoAsc1 - 1)
                
            
            return (pisosPasajeros, pasAsc1, pisoAsc1 - 1)
            
        nuevosPasajeros = [x for x in pasAsc1]
        accion = a.split()[0]
        index = int(a.split()[2])
        
        if (accion == "Subir"):
            nuevosPasajeros[index-1] = 1
            return (pisosPasajeros,tuple(nuevosPasajeros),pisoAsc1)
        
        if (accion == "Bajar"):
            nuevosPasajeros[index-1] = 0
            return (pisosPasajeros,tuple(nuevosPasajeros),pisoAsc1)
        
        
    def goal_test(self, s):
        pasAsc1 = s[1]
        
        ascensoresVacios = (sum(pasAsc1) == 0)
        
        pasajerosEnSuPiso = True
        
        for i in range(len(s[0])):
            pasajerosEnSuPiso = pasajerosEnSuPiso and (s[0][i] == self.goal[0][i])
        
        return pasajerosEnSuPiso and ascensoresVacios 
    
    def h1(self, node):
        sumaDistancias = 0
        
        pisosPasajeros = node.state[0]
        pisosObjetivos = self.goal[0]
        
        for x in range(len(pisosPasajeros)):
                sumaDistancias += abs(pisosPasajeros[x] - pisosObjetivos[x])
                
        return sumaDistancias

### Representacion:

((pisPas1, pisPas2), pasAsc1, pisoA1) -> ((2,4),(0,0),0) -> ((3,11),(0,0),0)

Donde pasAsc1 es una tupla que contiene 0 si el pasajero no esta dentro del ascensor y 1 si esta dentro.

Heuristica distancia entre origen y destino

In [20]:
estado_inicial_e2 = ((2,4,1,8),(0,0,0,0),0)
objetivo_e2 = ((3,11,12,1),(0,0,0,0),0)
problema_2 = ProblemaAscensoresV2(estado_inicial_e2, objetivo_e2)
resuelve_problema(problema_2, astar_search, problema_2.h1)

Solución:
	Paso 1: Subir
	Paso 2: Subir pasajero 3
	Paso 3: Subir
	Paso 4: Subir pasajero 1
	Paso 5: Subir
	Paso 6: Bajar pasajero 1
	Paso 7: Subir
	Paso 8: Subir pasajero 2
	Paso 9: Subir
	Paso 10: Subir
	Paso 11: Subir
	Paso 12: Subir
	Paso 13: Subir
	Paso 14: Subir
	Paso 15: Subir
	Paso 16: Bajar pasajero 2
	Paso 17: Subir
	Paso 18: Bajar pasajero 3
	Paso 19: Bajar
	Paso 20: Bajar
	Paso 21: Bajar
	Paso 22: Bajar
	Paso 23: Subir pasajero 4
	Paso 24: Bajar
	Paso 25: Bajar
	Paso 26: Bajar
	Paso 27: Bajar
	Paso 28: Bajar
	Paso 29: Bajar
	Paso 30: Bajar
	Paso 31: Bajar pasajero 4

Tiempo del algoritmo: 3.1946 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 31.
Nodos analizados: 2785


## Ejecución con todos los pasajeros

In [21]:
estado_inicial_e3 = ((2,4,1,8,1),(0,0,0,0,0),0)
objetivo_e3 = ((3,11,12,1,9),(0,0,0,0,0),0)
problema_3 = ProblemaAscensoresV2(estado_inicial_e3, objetivo_e3)
resuelve_problema(problema_3, astar_search, problema_3.h1)

Solución:
	Paso 1: Subir
	Paso 2: Subir pasajero 5
	Paso 3: Subir
	Paso 4: Subir
	Paso 5: Subir
	Paso 6: Subir pasajero 2
	Paso 7: Subir
	Paso 8: Subir
	Paso 9: Subir
	Paso 10: Subir
	Paso 11: Subir
	Paso 12: Bajar pasajero 2
	Paso 13: Bajar pasajero 5
	Paso 14: Bajar
	Paso 15: Subir pasajero 4
	Paso 16: Bajar
	Paso 17: Bajar
	Paso 18: Bajar
	Paso 19: Bajar
	Paso 20: Bajar
	Paso 21: Bajar
	Paso 22: Bajar
	Paso 23: Bajar pasajero 4
	Paso 24: Subir pasajero 3
	Paso 25: Subir
	Paso 26: Subir pasajero 1
	Paso 27: Subir
	Paso 28: Bajar pasajero 1
	Paso 29: Subir
	Paso 30: Subir
	Paso 31: Subir
	Paso 32: Subir
	Paso 33: Subir
	Paso 34: Subir
	Paso 35: Subir pasajero 2
	Paso 36: Subir
	Paso 37: Subir
	Paso 38: Bajar pasajero 2
	Paso 39: Subir
	Paso 40: Bajar pasajero 3

Tiempo del algoritmo: 1918.0537 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 40.
Nodos analizados: 55680


## Solución del problema generalizado en pasajeros con 2 ascensores:

### Representación:

((pisPas1, pisPas2), (pasAsc1, pasAsc2), pisoA1) -> ((2,4), ((0,0), (0,0)), (0,0) ) -> ((3,11), ((0,0), (0,0)), (0,0) )

Donde pasAsc1 y pasAsc2 son tuplas que contienen 0 si el pasajero no esta dentro del ascensor y 1 si esta dentro.

In [40]:
from search import *

class ProblemaAscensoresV3(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None, limAscLento=2):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self._limiteAscLento = limAscLento
    
    def pasDentroDeAsc(self,indexPas,pasajerosAsc):
        dentro = [(1 == pasAsc[indexPas]) for pasAsc in pasajerosAsc]
        return True in dentro

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        accs = list()
        
        pasajerosAsc = s[1]
        pasAsc1 = pasajerosAsc[0]
        pasAsc2 = pasajerosAsc[1]
        cantPasAsc1 = sum(pasAsc1)
        cantPasAsc2 = sum(pasAsc2)
        pisosAsc = s[2]
        pisoAsc1 = pisosAsc[0]
        pisoAsc2 = pisosAsc[1]
        pisosPasajeros = s[0]
        
        
        if (pisoAsc1 > 0):
            accs.append("Bajar ascensor 1")
        
        if (pisoAsc1 < 12):
            accs.append("Subir ascensor 1")
        
        if (pisoAsc2 > 0):
            accs.append("Bajar ascensor 2")
        
        if (pisoAsc2 < 12):
            accs.append("Subir ascensor 2")
        
        if ((pisoAsc1 > 0) and (pisoAsc2 > 0)):
            accs.append("Bajar ascensor 1 y 2")

        if ((pisoAsc1 < 12) and (pisoAsc2 < 12)):
            accs.append("Subir ascensor 1 y 2")
        
        if ((pisoAsc1 > 0) and (pisoAsc2 < 12)):
            accs.append("Bajar ascensor 1 y subir ascensor 2")
            
        if ((pisoAsc1 < 12) and (pisoAsc2 > 0)):
            accs.append("Subir ascensor 1 y bajar ascensor 2")
            
        #Esta parte del codigo esta generalizada para cualquier cantidad de pasajeros
        i = 1
        for pisoPas in pisosPasajeros:
            if ((pisoPas == pisoAsc1) and 
                (cantPasAsc1 < self._limiteAscLento) and
                not self.pasDentroDeAsc(i-1,pasajerosAsc)):
                accs.append("Subir pasajero " + str(i) + " al ascensor 1")
            if ((pisoPas == pisoAsc2) and 
                (cantPasAsc2 < self._limiteAscLento) and
                not self.pasDentroDeAsc(i-1,pasajerosAsc)):
                accs.append("Subir pasajero " + str(i) + " al ascensor 2")
            i += 1
        
        i = 1
        for pas in pasAsc1:
            if (pas == 1):
                accs.append("Bajar pasajero " + str(i) + " del ascensor 1")
            i += 1
        
        i = 1
        for pas in pasAsc2:
            if (pas == 1):
                accs.append("Bajar pasajero " + str(i) + " del ascensor 2")
            i += 1
        
        return accs
    
    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        
        #print("Accion aplicada: {A} \nEstado Actual: {S}".format(A=a,S=s))
        
                
        pasajerosAsc = s[1]
        pasAsc1 = pasajerosAsc[0]
        pasAsc2 = pasajerosAsc[1]
        cantPasAsc1 = sum(pasAsc1)
        cantPasAsc2 = sum(pasAsc2)
        pisosAsc = s[2]
        pisoAsc1 = pisosAsc[0]
        pisoAsc2 = pisosAsc[1]
        pisosPasajeros = s[0]
        
        if (a == "Subir ascensor 1"):
            if (1 in pasAsc1):
                dentroAsc1 = []
                
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                    
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc1:
                    nuevosPisos[index] += 1
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2))
        
        if (a == "Bajar ascensor 1"):
            if (1 in pasAsc1):
                dentroAsc1 = []
                
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc1:
                    nuevosPisos[index] -= 1
                    
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2))
        
        if (a == "Subir ascensor 2"):
            if (1 in pasAsc2):
                dentroAsc2 = []
                
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                    
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc2:
                    nuevosPisos[index] += 1
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1,pisoAsc2 + 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1,pisoAsc2 + 1))
        
        if (a == "Bajar ascensor 2"):
            if (1 in pasAsc2):
                dentroAsc2 = []
                
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                for index in dentroAsc2:
                    nuevosPisos[index] -= 1
                    
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1,pisoAsc2 - 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1,pisoAsc2 - 1))
        
        if (a == "Bajar ascensor 1 y 2"):
            if (1 in pasAsc1) or (1 in pasAsc2) :
                dentroAsc1 = []
                dentroAsc2 = []
                
                # Busco los pasajeros dentro del ascensor 1
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                # Busco los pasajeros dentro del ascensor 2
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                # Muevo los pasajeros que esten dentro del ascensor 1
                for index in dentroAsc1:
                    nuevosPisos[index] -= 1
                    
                # Muevo los pasajeros que esten dentro del ascensor 2
                for index in dentroAsc2:
                    nuevosPisos[index] -= 1
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2 - 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2 - 1))
        
        if (a == "Subir ascensor 1 y 2"):
            if (1 in pasAsc1) or (1 in pasAsc2) :
                dentroAsc1 = []
                dentroAsc2 = []
                
                # Busco los pasajeros dentro del ascensor 1
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                # Busco los pasajeros dentro del ascensor 2
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                # Muevo los pasajeros que esten dentro del ascensor 1
                for index in dentroAsc1:
                    nuevosPisos[index] += 1
                    
                # Muevo los pasajeros que esten dentro del ascensor 2
                for index in dentroAsc2:
                    nuevosPisos[index] += 1
                    
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2 + 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2 + 1))
        
        if (a == "Bajar ascensor 1 y subir ascensor 2"):
            if (1 in pasAsc1) or (1 in pasAsc2) :
                dentroAsc1 = []
                dentroAsc2 = []
                
                # Busco los pasajeros dentro del ascensor 1
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                # Busco los pasajeros dentro del ascensor 2
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                # Muevo los pasajeros que esten dentro del ascensor 1
                for index in dentroAsc1:
                    nuevosPisos[index] -= 1
                    
                # Muevo los pasajeros que esten dentro del ascensor 2
                for index in dentroAsc2:
                    nuevosPisos[index] += 1
                    
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2 + 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 - 1,pisoAsc2 + 1))
        
        if (a == "Subir ascensor 1 y bajar ascensor 2"):
            if (1 in pasAsc1) or (1 in pasAsc2) :
                dentroAsc1 = []
                dentroAsc2 = []
                
                # Busco los pasajeros dentro del ascensor 1
                i = 0
                for pas in pasAsc1:
                    if (pas == 1):
                        dentroAsc1.append(i)
                    i += 1
                
                # Busco los pasajeros dentro del ascensor 2
                i = 0
                for pas in pasAsc2:
                    if (pas == 1):
                        dentroAsc2.append(i)
                    i += 1
                
                nuevosPisos = [x for x in pisosPasajeros]
                # Muevo los pasajeros que esten dentro del ascensor 1
                for index in dentroAsc1:
                    nuevosPisos[index] += 1
                    
                # Muevo los pasajeros que esten dentro del ascensor 2
                for index in dentroAsc2:
                    nuevosPisos[index] -= 1
                    
                
                return (tuple(nuevosPisos), (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2 - 1))
            
            return (pisosPasajeros, (pasAsc1,pasAsc2) , (pisoAsc1 + 1,pisoAsc2 - 1))
        
        # Generalizado
        nuevosPasajeros = [[x for x in pasAsc1],[x for x in pasAsc2]]
        accion = a.split()[0]
        index = int(a.split()[2])
        indexAsc = int(a.split()[-1])
        
        if (accion == "Subir"):
            nuevosPasajeros[indexAsc-1][index-1] = 1
            tupla_pasajeros = tuple(tuple(x) for x in nuevosPasajeros)
            return (pisosPasajeros,tupla_pasajeros,pisosAsc)
        
        if (accion == "Bajar"):
            nuevosPasajeros[indexAsc-1][index-1] = 0
            tupla_pasajeros = tuple(tuple(x) for x in nuevosPasajeros)
            return (pisosPasajeros,tupla_pasajeros,pisosAsc)
        
        
    def goal_test(self, s):
        pasAsc = s[1]
        pasAsc1 = pasAsc[0]
        pasAsc2 = pasAsc[1]
        
        ascensoresVacios = (sum(pasAsc1) == 0) and (sum(pasAsc2) == 0)
        
        pasajerosEnSuPiso = True
        
        for i in range(len(s[0])):
            pasajerosEnSuPiso = pasajerosEnSuPiso and (s[0][i] == self.goal[0][i])
        
        return pasajerosEnSuPiso and ascensoresVacios 
    
    def h1(self, node):
        sumaDistancias = 0
        
        pisosPasajeros = node.state[0]
        pisosObjetivos = self.goal[0]
        
        for x in range(len(pisosPasajeros)):
                sumaDistancias += abs(pisosPasajeros[x] - pisosObjetivos[x])
                
        return sumaDistancias
            

In [42]:
estado_inicial_e4 = ((2,4,1),((0,0,0),(0,0,0)),(0,0))
objetivo_e4 = ((3,11,12),((0,0,0),(0,0,0)),(0,0))
problema_4 = ProblemaAscensoresV3(estado_inicial_e4, objetivo_e4)
resuelve_problema(problema_4, astar_search, problema_4.h1)

Solución:
	Paso 1: Subir ascensor 2
	Paso 2: Subir pasajero 3 al ascensor 2
	Paso 3: Subir ascensor 2
	Paso 4: Subir ascensor 2
	Paso 5: Subir ascensor 2
	Paso 6: Subir pasajero 2 al ascensor 2
	Paso 7: Subir ascensor 2
	Paso 8: Subir ascensor 2
	Paso 9: Subir ascensor 2
	Paso 10: Subir ascensor 2
	Paso 11: Subir ascensor 2
	Paso 12: Subir ascensor 1 y 2
	Paso 13: Subir ascensor 1 y 2
	Paso 14: Bajar pasajero 2 del ascensor 2
	Paso 15: Subir pasajero 1 al ascensor 1
	Paso 16: Subir ascensor 1 y 2
	Paso 17: Bajar pasajero 1 del ascensor 1
	Paso 18: Bajar pasajero 3 del ascensor 2

Tiempo del algoritmo: 0.9692 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 18.
Nodos analizados: 526


### Ejecucion del problema anterior con un solo ascensor

Procedemos a buscar una solución al estado anterior utilizando un solo ascensor para posteriormente realizar comparaciones:

In [43]:
estado_inicial_e2 = ((2,4,1),(0,0,0),0)
objetivo_e2 = ((3,11,12),(0,0,0),0)
problema_2 = ProblemaAscensoresV2(estado_inicial_e2, objetivo_e2)
resuelve_problema(problema_2, astar_search, problema_2.h1)

Solución:
	Paso 1: Subir
	Paso 2: Subir pasajero 3
	Paso 3: Subir
	Paso 4: Subir pasajero 1
	Paso 5: Subir
	Paso 6: Bajar pasajero 1
	Paso 7: Subir
	Paso 8: Subir pasajero 2
	Paso 9: Subir
	Paso 10: Subir
	Paso 11: Subir
	Paso 12: Subir
	Paso 13: Subir
	Paso 14: Subir
	Paso 15: Subir
	Paso 16: Bajar pasajero 2
	Paso 17: Subir
	Paso 18: Bajar pasajero 3

Tiempo del algoritmo: 0.0719 segundos
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 18.
Nodos analizados: 454


In [45]:
estado_inicial_e5 = ((2,4,1,8),((0,0,0,0),(0,0,0,0)),(0,0))
objetivo_e5 = ((3,11,12,1),((0,0,0,0),(0,0,0,0)),(0,0))
problema_5 = ProblemaAscensoresV3(estado_inicial_e5, objetivo_e5)
resuelve_problema(problema_5, astar_search, problema_5.h1)

Solución:
	Paso 1: Subir ascensor 1 y 2
	Paso 2: Subir pasajero 3 al ascensor 1
	Paso 3: Subir ascensor 1 y 2
	Paso 4: Subir ascensor 1 y 2
	Paso 5: Subir ascensor 1 y 2
	Paso 6: Subir pasajero 2 al ascensor 2
	Paso 7: Subir ascensor 1 y 2
	Paso 8: Subir ascensor 1 y 2
	Paso 9: Subir ascensor 1 y 2
	Paso 10: Subir ascensor 1 y 2
	Paso 11: Subir pasajero 4 al ascensor 1
	Paso 12: Bajar pasajero 3 del ascensor 1
	Paso 13: Bajar ascensor 1 y subir ascensor 2
	Paso 14: Bajar ascensor 1 y subir ascensor 2
	Paso 15: Bajar ascensor 1 y subir ascensor 2
	Paso 16: Bajar pasajero 2 del ascensor 2
	Paso 17: Bajar ascensor 1 y 2
	Paso 18: Bajar ascensor 1 y 2
	Paso 19: Bajar ascensor 1 y 2
	Paso 20: Subir pasajero 3 al ascensor 2
	Paso 21: Bajar ascensor 1 y subir ascensor 2
	Paso 22: Bajar pasajero 4 del ascensor 1
	Paso 23: Subir ascensor 1 y 2
	Paso 24: Subir pasajero 1 al ascensor 1
	Paso 25: Subir ascensor 1 y 2
	Paso 26: Bajar pasajero 1 del ascensor 1
	Paso 27: Bajar ascensor 1 y subir asce

### Comparación de resultados

Como podemos observar trabajando con la misma heuristica (`h1`) y el mismo algoritmo (`A*`) nos ha dado como resultado:

| VERSION DEL PROBLEMA | TIEMPO (s) | LONGITUD DE LA SOLUCIÓN | NODOS ANALIZADOS |
| -------------------- | ---------- | ----------------------- | ---------------- |
| <center>1 ascensor y 3 pasajeros</center> | <center>0.0719</center> | <center>18</center> | <center>454</center> |
| <center>2 ascensores y 3 pasajeros</center> | <center>0.9692</center> | <center>18</center> | <center>526</center> |
| <center>1 ascensor y 4 pasajeros</center> | <center>2.9931</center> | <center>31</center> | <center>2785</center> |
| <center>2 ascensores y 4 pasajeros</center> | <center>992.7739</center> | <center>28</center> | <center>17059</center> |

Dados estos resultados podemos ver que para este problema en especifico podemos utilizar un solo ascensor si queremos que el tiempo de busqueda de una solucion sea muy rapido y si lo que queremos es hacer uso total de los ascensores tendremos que estar dispuestos a sacrificar el tiempo.

## Solución del problema generalizado en pasajeros y ascensores

Se procedera a ampliar los operadores de la implementación anterior para que cumpla con lo que indica la practica.

In [2]:
import numpy as np
from search import *

class ProblemaAscensoresV4(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, 
                 goal=None, 
                 limAscLento=2, 
                 limAscRapido=3, 
                 cantAscRapidos=1, 
                 velAscRapido=2,
                 bloques=((0,12),(0,12),(0,12),(0,12),(0,12)) ):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self._limiteAscLento = limAscLento
        self._limiteAscRapido = limAscRapido
        self._limAsc = bloques
        self._cantAscLentos = len(bloques) - cantAscRapidos
        self._cantAscRapidos = cantAscRapidos
        self._velAscRapido = velAscRapido
        aux = list()
        for bloque in bloques:
            if bloque not in aux:
                aux.append(bloque)
        self._bloques = tuple(aux)
        
    def path_cost(self, c, state1, action, state2):
        return c + 1
        """
        newCost = 0
        acc = action.split()[2:]
        if "ASCENSORES" in action:
            for a in acc:
                movimiento, index_asc = a.split(".")
                if "(RAPIDO)" in movimiento:
                    newCost += 1
                else:
                    newCost += self._velAscRapido
        else:
            newCost += 1
            
        return c + newCost
        """
    
    def pasDentroDeAsc(self,indexPas,pasajerosAsc):
        dentro = [(1 == pasAsc[indexPas]) for pasAsc in pasajerosAsc]
        return True in dentro
    
    def accionesAscensores(self, prefix, numAsc, control, accs, estado_actual):
        '''
        Devuelve las acciones posibles con un numero de accensores.
        '''
        if (numAsc == 0):
            accs.append(prefix)
        else:
            es_rapido = "_(RAPIDO)" if (control > len(estado_actual[2]) - self._cantAscRapidos) else ""
            # SUBIR EL ASCENSOR
            if (estado_actual[2][control-1] < self._limAsc[control-1][1]):
                self.accionesAscensores(prefix + " SUBIR_ASCENSOR" + es_rapido + "." + str(control), numAsc - 1, control + 1, accs, estado_actual)
            # BAJAR EL ASCENSOR
            if (estado_actual[2][control-1] > self._limAsc[control-1][0]):
                self.accionesAscensores(prefix + " BAJAR_ASCENSOR" + es_rapido + "." + str(control), numAsc - 1, control + 1, accs, estado_actual)
            # NO APLICAR MOVIMIENTO AL ASCENSOR
            if (len(prefix.split()) > 2):
                self.accionesAscensores(prefix, numAsc - 1, control + 1, accs, estado_actual)
        return accs

    def resultadoAccionAsc(self, accion, estado_actual):
        '''
        Devuelve el resultado de aplicar la accion al estado actual
        '''
        nuevo_estado = [list(estado_actual[0]), [list(x) for x in estado_actual[1]], list(estado_actual[2])]
        acc = accion.split()[2:]
        indexAsc = 1
        for a in acc:
            movimiento, index_asc = a.split(".")
            dentroAsc = []
            cantPisos = self._velAscRapido if ("(RAPIDO)" in a) else 1

            i = 0
            for pas in nuevo_estado[1][int(index_asc)-1]:
                if (pas == 1):
                    dentroAsc.append(i)
                i += 1

            if "SUBIR" in movimiento:
                for index in dentroAsc:
                    nuevo_estado[0][index] += cantPisos
                nuevo_estado[2][int(index_asc)-1] += cantPisos

            if "BAJAR" in movimiento:
                for index in dentroAsc:
                    nuevo_estado[0][index] -= cantPisos
                nuevo_estado[2][int(index_asc)-1] -= cantPisos

        return (tuple(nuevo_estado[0]), estado_actual[1], tuple(nuevo_estado[2]))

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        accs = list()
        pisosPasajeros = s[0]
        pasajerosAsc = s[1]
        pisosAsc = s[2]
        
        numAsc = len(pisosAsc)
        prefix = "ACCIONES ASCENSORES:"
        # FUNCIÓN RECURSIVA QUE DEVUELVE TODAS LASPOSIBLES ACIONES CON LOS ASCENSORES DADOS
        accs = self.accionesAscensores(prefix, numAsc, 1, [], s)

        # ACCIONES DE LOS PASAJEROS
        #Esta parte del codigo esta generalizada para cualquier cantidad de pasajeros
        # SUBIR PASAJERO A UN ASCENSOR
        indexPas = 1
        for pisoPas in pisosPasajeros:
            for indexAsc in range(len(pisosAsc)):
                if ((pisoPas == pisosAsc[indexAsc]) and 
                    (pisoPas != self.goal[0][indexPas-1]) and
                    (sum(pasajerosAsc[indexAsc]) < self._limiteAscLento) and
                    not self.pasDentroDeAsc(indexPas-1,pasajerosAsc)):
                    accs.append("Subir pasajero {PAS} al ascensor {ASC}".format(PAS=indexPas,ASC=indexAsc+1))
            indexPas += 1
        
        # BAJAR PASAJERO DE UN ASCENSOR
        indexAsc = 1
        for pasAsc in pasajerosAsc:
            indexPas = 1
            for pas in pasAsc:
                if (pas == 1):
                    accs.append("Bajar pasajero {PAS} del ascensor {ASC}".format(PAS=indexPas, ASC=indexAsc))
                indexPas += 1
            indexAsc += 1
        
        return accs
    
    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        
        #print("Accion aplicada: {A} \nEstado Actual: {S}".format(A=a,S=s))
    
        pasajerosAsc = s[1]
        pisosAsc = s[2]
        pisosPasajeros = s[0]
        
        if ("ASCENSORES" in a):
            return self.resultadoAccionAsc(a,s)
        else:
            # SI LA ACCION NO CONTIENE LA PALABRA ASCENSORES ES PORQUE SE DESEA MOVER UN PASAJERO
            # Generalizado
            nuevosPasajeros = [ list(pas) for pas in pasajerosAsc ]
            aux = a.split()
            accion = aux[0]
            index = int(aux[2])
            indexAsc = int(aux[-1])
            
            if (accion == "Subir"):
                nuevosPasajeros[indexAsc-1][index-1] = 1
                tupla_pasajeros = tuple(tuple(x) for x in nuevosPasajeros)
                return (pisosPasajeros,tupla_pasajeros,pisosAsc)
            
            if (accion == "Bajar"):
                nuevosPasajeros[indexAsc-1][index-1] = 0
                tupla_pasajeros = tuple(tuple(x) for x in nuevosPasajeros)
                return (pisosPasajeros,tupla_pasajeros,pisosAsc)
        
        
    def goal_test(self, s):
        """
        Comprueba que todos los pasajeros esten en su piso que nadie se quede
        dentro de un ascensor.
        """
        # COMPROBAMOS QUE EN LOS ASCENSORES NO SE QUEDE NADIE
        ascensoresVacios = sum(sum(x) for x in s[1]) == 0
        if (not ascensoresVacios): return False

        # COMPROBAMOS QUE TODOS LOS PASAJEROS ESTEN EN SU PISO
        pasajerosEnSuPiso = True
        for i in range(len(s[0])):
            pasajerosEnSuPiso = pasajerosEnSuPiso and (s[0][i] == self.goal[0][i])
            if (not pasajerosEnSuPiso):
                return False
        
        return pasajerosEnSuPiso and ascensoresVacios 
    
    def h1(self, node):
        '''
        Devuelve la distancia entre cada pasajero y su piso final
        '''
        sumaDistancias = 0
        
        pisosPasajeros = node.state[0]
        pisosObjetivos = self.goal[0]
        
        for x in range(len(pisosPasajeros)):
                sumaDistancias += abs(pisosPasajeros[x] - pisosObjetivos[x])
                
        return sumaDistancias

    def buscar_mas_cercano(self,tupla,valor):
        """
            Se pasa una tupla y el valor que se quiere comparar
        """
        array = np.array(tupla)
        idx = (np.abs(array-valor)).argmin()
        return array[idx], idx
    
    def distanciaAscensorYPasajero(self, pisoPas, pisosAsc):
        pisoAscMasCercano, index = self.buscar_mas_cercano(pisosAsc, pisoPas)
        return abs(pisoPas - pisoAscMasCercano), index
        
    def buscar_persona_dentro_asc(self,pasajerosAsc, indexPas):
        aux = np.array(pasajerosAsc)
        result = np.where(aux == 1)
        return indexPas in result[1]
    
    def costeSubirBajar(self, pisoAct, pisoObj, pasajerosAsc, indexPas):       
        if (pisoAct == pisoObj):
            if (self.buscar_persona_dentro_asc(pasajerosAsc,indexPas)):
                return 1
            return 0
        else:
            return 2

    def h2(self, node):
        '''
        Devuelve la distancia entre el ascensor mas cercano y el pasajero sumada a
        la distancia entre cada pasajero y su piso final (h1)
        '''
        
        distanciaPasObj = 0        
        distanciaAscPas = 0
        costeSubirBajarPas = 0
        pisosPasajeros = node.state[0]
        pisosObjetivos = self.goal[0]      
        pasajerosAsc = node.state[1]
        pisosAsc = node.state[2]

        for indexPas in range(len(pisosPasajeros)):
            # Calculamos la distancia entre cada pasajero y su piso objetivo.
            distanciaPasObj += abs(pisosPasajeros[indexPas] - pisosObjetivos[indexPas])
            # Calculamos la distancia entre cada pasajero y su ascensor mas cercano.
            coste, idxAsc = self.distanciaAscensorYPasajero(pisosPasajeros[indexPas],pisosAsc)
            distanciaAscPas += coste
            # Calculamos el coste de subir y bajar a los pasajeros
            costeSubirBajarPas += self.costeSubirBajar(pisosPasajeros[indexPas], pisosObjetivos[indexPas], pasajerosAsc, indexPas)        
        
        h = distanciaPasObj + distanciaAscPas + costeSubirBajarPas
        
        return h

# Problema con 1 ascensor y 1 persona

In [45]:
pisosAsc = (0,)
pisosPasInicial = (2,)
pisosPasFinal = (3,)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),)
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir ascensor 1
	Paso 3: Subir pasajero 1 al ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Bajar pasajero 1 del ascensor 1

Tiempo del algoritmo: 0.0005 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 5.
Nodos analizados: 6


# Problema con 1 ascensor y 2 personas

In [46]:
pisosAsc = (0,)
pisosPasInicial = (2,4)
pisosPasFinal = (3,11)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),)
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir ascensor 1
	Paso 3: Subir pasajero 1 al ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Bajar pasajero 1 del ascensor 1
	Paso 6: Subir ascensor 1
	Paso 7: Subir pasajero 2 al ascensor 1
	Paso 8: Subir ascensor 1
	Paso 9: Subir ascensor 1
	Paso 10: Subir ascensor 1
	Paso 11: Subir ascensor 1
	Paso 12: Subir ascensor 1
	Paso 13: Subir ascensor 1
	Paso 14: Subir ascensor 1
	Paso 15: Bajar pasajero 2 del ascensor 1

Tiempo del algoritmo: 0.0172 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 15.
Nodos analizados: 107


# Problema con 1 ascensor y 3 personas

In [47]:
pisosAsc = (0,)
pisosPasInicial = (2,4,1)
pisosPasFinal = (3,11,12)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),)
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir pasajero 3 al ascensor 1
	Paso 3: Subir ascensor 1
	Paso 4: Subir pasajero 1 al ascensor 1
	Paso 5: Subir ascensor 1
	Paso 6: Bajar pasajero 1 del ascensor 1
	Paso 7: Subir ascensor 1
	Paso 8: Subir pasajero 2 al ascensor 1
	Paso 9: Subir ascensor 1
	Paso 10: Subir ascensor 1
	Paso 11: Subir ascensor 1
	Paso 12: Subir ascensor 1
	Paso 13: Subir ascensor 1
	Paso 14: Subir ascensor 1
	Paso 15: Subir ascensor 1
	Paso 16: Bajar pasajero 2 del ascensor 1
	Paso 17: Subir ascensor 1
	Paso 18: Bajar pasajero 3 del ascensor 1

Tiempo del algoritmo: 0.0060 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 18.
Nodos analizados: 28


# Problema con 1 ascensor y 4 personas

In [48]:
pisosAsc = (0,)
pisosPasInicial = (2,4,1,8)
pisosPasFinal = (3,11,12,1)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),)
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir pasajero 3 al ascensor 1
	Paso 3: Subir ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Subir ascensor 1
	Paso 6: Subir pasajero 2 al ascensor 1
	Paso 7: Subir ascensor 1
	Paso 8: Subir ascensor 1
	Paso 9: Subir ascensor 1
	Paso 10: Subir ascensor 1
	Paso 11: Subir ascensor 1
	Paso 12: Subir ascensor 1
	Paso 13: Subir ascensor 1
	Paso 14: Bajar pasajero 2 del ascensor 1
	Paso 15: Subir ascensor 1
	Paso 16: Bajar pasajero 3 del ascensor 1
	Paso 17: Bajar ascensor 1
	Paso 18: Bajar ascensor 1
	Paso 19: Bajar ascensor 1
	Paso 20: Bajar ascensor 1
	Paso 21: Subir pasajero 4 al ascensor 1
	Paso 22: Bajar ascensor 1
	Paso 23: Bajar ascensor 1
	Paso 24: Bajar ascensor 1
	Paso 25: Bajar ascensor 1
	Paso 26: Bajar ascensor 1
	Paso 27: Bajar ascensor 1
	Paso 28: Bajar ascensor 1
	Paso 29: Bajar pasajero 4 del ascensor 1
	Paso 30: Subir ascensor 1
	Paso 31: Subir pasajero 1 al ascensor 1
	Paso 32: Subir ascensor 1
	Paso 33: Bajar pasajero 1 del asc

Este es el ejemplo perfecto para darnos cuenta de que la heuristica en lugar de informar al problema lo esta desinformando.

# Problema con 1 ascensor y 5 personas

In [29]:
pisosAsc = (0,)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,11,12,1,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),)
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h1)

Hemos detenido la ejecución del problema ya que al no tener una buena heurística se tarda mucho en encontrar una solución.

## Problema con 2 ascensores y 1 persona

In [49]:
pisosAsc = (0,0)
pisosPasInicial = (2,)
pisosPasFinal = (3,)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)

resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor 2
	Paso 2: Bajar ascensor 1, subir ascensor 2
	Paso 3: Subir pasajero 1 al ascensor 2
	Paso 4: Subir ascensor 1, subir ascensor 2
	Paso 5: Bajar pasajero 1 del ascensor 2

Tiempo del algoritmo: 0.0026 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 5.
Nodos analizados: 13


## Problema con 2 ascensores y 2 personas

In [50]:
pisosAsc = (0,0)
pisosPasInicial = (2,4)
pisosPasFinal = (3,11)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)

resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor 2
	Paso 2: Subir ascensor 1, subir ascensor 2
	Paso 3: Subir pasajero 1 al ascensor 2
	Paso 4: Subir ascensor 1, subir ascensor 2
	Paso 5: Bajar pasajero 1 del ascensor 2
	Paso 6: Subir ascensor 1
	Paso 7: Subir pasajero 2 al ascensor 1
	Paso 8: Subir ascensor 1
	Paso 9: Subir ascensor 1
	Paso 10: Subir ascensor 1
	Paso 11: Subir ascensor 1
	Paso 12: Subir ascensor 1
	Paso 13: Subir ascensor 1
	Paso 14: Subir ascensor 1
	Paso 15: Bajar pasajero 2 del ascensor 1

Tiempo del algoritmo: 0.0242 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 15.
Nodos analizados: 51


## Problema con 2 ascensores y 3 personas

In [51]:
pisosAsc = (0,0)
pisosPasInicial = (2,4,1)
pisosPasFinal = (3,11,12)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)

resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir ascensor 1, subir ascensor 2
	Paso 3: Subir pasajero 3 al ascensor 2
	Paso 4: Subir ascensor 1, subir ascensor 2
	Paso 5: Subir pasajero 1 al ascensor 2
	Paso 6: Subir ascensor 1, subir ascensor 2
	Paso 7: Subir pasajero 2 al ascensor 1
	Paso 8: Bajar pasajero 1 del ascensor 2
	Paso 9: Subir ascensor 1, subir ascensor 2
	Paso 10: Subir ascensor 1, subir ascensor 2
	Paso 11: Subir ascensor 1, subir ascensor 2
	Paso 12: Subir ascensor 1, subir ascensor 2
	Paso 13: Subir ascensor 1, subir ascensor 2
	Paso 14: Subir ascensor 1, subir ascensor 2
	Paso 15: Subir ascensor 1, subir ascensor 2
	Paso 16: Bajar pasajero 2 del ascensor 1
	Paso 17: Bajar ascensor 1, subir ascensor 2
	Paso 18: Bajar ascensor 1, subir ascensor 2
	Paso 19: Bajar pasajero 3 del ascensor 2

Tiempo del algoritmo: 0.1299 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 19.
Nodos analizados: 120


# Problema con 2 ascensores y 4 personas

In [5]:
pisosAsc = (0,0)
pisosPasInicial = (2,4,1,8)
pisosPasFinal = (3,11,12,1)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)

resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir ascensor 1, subir ascensor 2
	Paso 3: Subir ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Subir pasajero 3 al ascensor 2
	Paso 6: Subir ascensor 1, subir ascensor 2
	Paso 7: Subir ascensor 1, subir ascensor 2
	Paso 8: Subir ascensor 1, subir ascensor 2
	Paso 9: Subir ascensor 1
	Paso 10: Subir pasajero 4 al ascensor 1
	Paso 11: Bajar ascensor 1
	Paso 12: Bajar ascensor 1
	Paso 13: Bajar ascensor 1
	Paso 14: Bajar ascensor 1, subir ascensor 2
	Paso 15: Bajar ascensor 1, subir ascensor 2
	Paso 16: Bajar ascensor 1, bajar ascensor 2
	Paso 17: Bajar ascensor 1, bajar ascensor 2
	Paso 18: Bajar pasajero 4 del ascensor 1
	Paso 19: Subir pasajero 2 al ascensor 2
	Paso 20: Subir ascensor 1, subir ascensor 2
	Paso 21: Bajar ascensor 1, subir ascensor 2
	Paso 22: Subir ascensor 1, subir ascensor 2
	Paso 23: Bajar ascensor 1, subir ascensor 2
	Paso 24: Subir ascensor 1, subir ascensor 2
	Paso 25: Bajar ascensor 1, subir ascensor 2
	Paso 26: Subir

# Problema con 2 ascensores y 5 personas

In [7]:
pisosAsc = (0,0)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,11,12,1,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir ascensor 1, subir ascensor 2
	Paso 3: Subir ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Subir pasajero 2 al ascensor 1
	Paso 6: Subir ascensor 1
	Paso 7: Subir ascensor 1
	Paso 8: Subir ascensor 1
	Paso 9: Subir ascensor 1
	Paso 10: Subir pasajero 5 al ascensor 2
	Paso 11: Subir pasajero 3 al ascensor 2
	Paso 12: Subir ascensor 1, subir ascensor 2
	Paso 13: Bajar ascensor 1, subir ascensor 2
	Paso 14: Subir ascensor 1, subir ascensor 2
	Paso 15: Bajar ascensor 1, subir ascensor 2
	Paso 16: Subir ascensor 1, subir ascensor 2
	Paso 17: Subir ascensor 1, subir ascensor 2
	Paso 18: Subir ascensor 1, subir ascensor 2
	Paso 19: Bajar pasajero 2 del ascensor 1
	Paso 20: Bajar ascensor 1, subir ascensor 2
	Paso 21: Bajar pasajero 5 del ascensor 2
	Paso 22: Bajar ascensor 1, subir ascensor 2
	Paso 23: Bajar ascensor 1, subir ascensor 2
	Paso 24: Subir pasajero 4 al ascensor 1
	Paso 25: Bajar ascensor 1, subir ascensor 2
	Paso 26: Bajar ascens

## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 1 persona y 2 bloques 0-6 y 6-12:

In [8]:
pisosAsc = (0,6,0)
pisosPasInicial = (2,)
pisosPasFinal = (3,)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,6),(6,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor (rapido) 3
	Paso 2: Subir ascensor 1, bajar ascensor (rapido) 3
	Paso 3: Subir pasajero 1 al ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Bajar pasajero 1 del ascensor 1

Tiempo del algoritmo: 0.0302 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 5.
Nodos analizados: 34


## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 2 personas y 2 bloques 0-6 y 6-12:

In [22]:
pisosAsc = (0,6,0)
pisosPasInicial = (2,4)
pisosPasFinal = (3,11)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,6),(6,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor (rapido) 3
	Paso 2: Subir ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 3: Subir pasajero 2 al ascensor 3
	Paso 4: Bajar ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 5: Subir ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 6: Bajar ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 7: Bajar pasajero 2 del ascensor 3
	Paso 8: Subir pasajero 2 al ascensor 2
	Paso 9: Subir ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 10: Bajar pasajero 2 del ascensor 2
	Paso 11: Subir pasajero 1 al ascensor 1
	Paso 12: Subir ascensor 1, bajar ascensor (rapido) 3
	Paso 13: Bajar pasajero 1 del ascensor 1

Tiempo del algoritmo: 2.3461 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 13.
Nodos analizados: 337


## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 3 personas y 2 bloques 0-6 y 6-12:

In [23]:
pisosAsc = (0,6,0)
pisosPasInicial = (2,4,1)
pisosPasFinal = (3,11,12)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,6),(6,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor (rapido) 3
	Paso 2: Subir ascensor 1, subir ascensor (rapido) 3
	Paso 3: Subir pasajero 2 al ascensor 3
	Paso 4: Bajar ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 5: Subir ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 6: Bajar ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 7: Bajar pasajero 2 del ascensor 3
	Paso 8: Subir pasajero 3 al ascensor 1
	Paso 9: Subir ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 10: Subir pasajero 1 al ascensor 1
	Paso 11: Subir ascensor 1, bajar ascensor (rapido) 3
	Paso 12: Bajar pasajero 1 del ascensor 1
	Paso 13: Subir pasajero 2 al ascensor 2
	Paso 14: Subir ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 15: Bajar pasajero 2 del ascensor 2
	Paso 16: Bajar pasajero 3 del ascensor 1
	Paso 17: Subir pasajero 3 al ascensor 3
	Paso 18: Bajar ascensor 1, subir ascensor (rapido) 3
	Paso 19: Bajar ascensor 1, subir ascensor (rapido

## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 4 personas y 2 bloques 0-6 y 6-12:

In [24]:
pisosAsc = (0,6,0)
pisosPasInicial = (2,4,1,8)
pisosPasFinal = (3,11,12,1)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,6),(6,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 2: Subir ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 3: Subir pasajero 2 al ascensor 3
	Paso 4: Bajar ascensor 1, subir ascensor (rapido) 3
	Paso 5: Subir ascensor 1, bajar ascensor 2, subir ascensor (rapido) 3
	Paso 6: Bajar ascensor 1, subir ascensor 2, subir ascensor (rapido) 3
	Paso 7: Subir pasajero 3 al ascensor 1
	Paso 8: Subir ascensor 1
	Paso 9: Bajar pasajero 2 del ascensor 3
	Paso 10: Subir pasajero 1 al ascensor 1
	Paso 11: Subir ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 12: Bajar pasajero 1 del ascensor 1
	Paso 13: Subir pasajero 4 al ascensor 3
	Paso 14: Subir ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 15: Subir ascensor 1, bajar ascensor (rapido) 3
	Paso 16: Subir pasajero 2 al ascensor 2
	Paso 17: Bajar ascensor 1, subir ascensor 2, bajar ascensor (rapido) 3
	Paso 18: Bajar pasajero 2 del ascensor 2
	Paso 19: Bajar pasajero 3 de

## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 5 personas y 2 bloques 0-6 y 0-12:

In [45]:
pisosAsc = (0,6,0)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,12,11,2,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,6),(6,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar este problema para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 3 ascensores (2 lentos y 1 rápido en pares), 5 personas y sin bloques:

In [105]:
pisosAsc = (0,4,8)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,11,12,1,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,12),(0,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=0
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1
	Paso 2: Subir pasajero 4 al ascensor 3
	Paso 3: Subir ascensor 1, bajar ascensor 3
	Paso 4: Bajar ascensor 1, bajar ascensor 3
	Paso 5: Subir ascensor 1, bajar ascensor 3
	Paso 6: Bajar ascensor 1, bajar ascensor 2, bajar ascensor 3
	Paso 7: Subir ascensor 1, subir ascensor 2, bajar ascensor 3
	Paso 8: Bajar ascensor 1, bajar ascensor 3
	Paso 9: Subir ascensor 1, bajar ascensor 3
	Paso 10: Bajar pasajero 4 del ascensor 3
	Paso 11: Subir pasajero 5 al ascensor 3
	Paso 12: Subir pasajero 3 al ascensor 3
	Paso 13: Bajar ascensor 1, subir ascensor 3
	Paso 14: Subir ascensor 1, subir ascensor 3
	Paso 15: Bajar ascensor 1, bajar ascensor 2, subir ascensor 3
	Paso 16: Subir ascensor 1, subir ascensor 2, subir ascensor 3
	Paso 17: Bajar ascensor 1, subir ascensor 3
	Paso 18: Subir ascensor 1, subir ascensor 3
	Paso 19: Bajar ascensor 1, subir ascensor 3
	Paso 20: Subir ascensor 1, subir ascensor 3
	Paso 21: Bajar pasajero 5 del ascensor 3
	Paso 22: Subir pa

## Problema con 4 ascensores (3 lentos y 1 rápido en pares), 1 persona y 3 bloques 0-4, 4-8 y 8-12:

In [67]:
pisosAsc = (0,4,8,0)
pisosPasInicial = (2,)
pisosPasFinal = (3,)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor (rapido) 4
	Paso 2: Subir ascensor 1, bajar ascensor (rapido) 4
	Paso 3: Subir pasajero 1 al ascensor 1
	Paso 4: Subir ascensor 1
	Paso 5: Bajar pasajero 1 del ascensor 1

Tiempo del algoritmo: 0.7194 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 5.
Nodos analizados: 98


## Problema con 4 ascensores (3 lentos y 1 rápido en pares), 2 personas y 3 bloques 0-4, 4-8 y 8-12:

In [107]:
pisosAsc = (0,4,8,0)
pisosPasInicial = (2,4)
pisosPasFinal = (3,11)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, subir ascensor (rapido) 4
	Paso 2: Subir ascensor 1, subir ascensor (rapido) 4
	Paso 3: Subir pasajero 2 al ascensor 4
	Paso 4: Bajar ascensor 1, subir ascensor (rapido) 4
	Paso 5: Subir ascensor 1, subir ascensor 3, subir ascensor (rapido) 4
	Paso 6: Subir pasajero 1 al ascensor 1
	Paso 7: Subir ascensor 1, subir ascensor 3, subir ascensor (rapido) 4
	Paso 8: Bajar pasajero 2 del ascensor 4
	Paso 9: Bajar pasajero 1 del ascensor 1
	Paso 10: Subir pasajero 2 al ascensor 3
	Paso 11: Bajar ascensor 1, subir ascensor 3, bajar ascensor (rapido) 4
	Paso 12: Bajar pasajero 2 del ascensor 3

Tiempo del algoritmo: 74.7801 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 12.
Nodos analizados: 1193


## Problema con 4 ascensores (3 lentos y 1 rápido en pares), 3 personas y 3 bloques 0-4, 4-8 y 8-12:

Para tener un tiempo de solución mejor, hemos dicho que los ascensores estan en los pisos donde hay pasajeros

In [52]:
pisosAsc = (2,8,10,0)
pisosPasInicial = (2,4,0)
pisosPasFinal = (3,11,12)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir pasajero 3 al ascensor 4
	Paso 2: Subir ascensor 1, bajar ascensor 2, bajar ascensor 3, subir ascensor (rapido) 4
	Paso 3: Bajar ascensor 1, bajar ascensor 2, bajar ascensor 3, subir ascensor (rapido) 4
	Paso 4: Subir pasajero 2 al ascensor 4
	Paso 5: Bajar ascensor 1, bajar ascensor 2, subir ascensor (rapido) 4
	Paso 6: Subir ascensor 1, bajar ascensor 2, subir ascensor (rapido) 4
	Paso 7: Bajar ascensor 1, subir ascensor (rapido) 4
	Paso 8: Bajar pasajero 2 del ascensor 4
	Paso 9: Subir ascensor 1, subir ascensor 3, subir ascensor (rapido) 4
	Paso 10: Bajar pasajero 3 del ascensor 4
	Paso 11: Subir pasajero 1 al ascensor 1
	Paso 12: Subir ascensor 1, subir ascensor 3
	Paso 13: Subir pasajero 2 al ascensor 3
	Paso 14: Bajar pasajero 1 del ascensor 1
	Paso 15: Bajar ascensor 1, subir ascensor 3
	Paso 16: Bajar pasajero 2 del ascensor 3

Tiempo del algoritmo: 40.2471 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 16.
Nodos analizados: 1

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible, por ese motivo hemos cambiado el piso de origin del 3er pasajero.

## Problema con 4 ascensores (3 lentos y 1 rápido en pares), 4 personas y 3 bloques 0-4, 4-8 y 8-12:

In [55]:
pisosAsc = (2,8,10,0)
pisosPasInicial = (2,4,1,8)
pisosPasFinal = (3,11,12,1)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 4 ascensores (3 lentos y 1 rápido en pares), 5 personas y 3 bloques 0-4, 4-8 y 8-12:

In [29]:
pisosAsc = (2,8,10,0)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,11,12,1,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 5 ascensores (4 lentos y 1 rápido en pares), 1 persona y 3 bloques 0-4, 4-8 y 8-12:

In [58]:
pisosAsc = (0,4,8,8,0)
pisosPasInicial = (2,)
pisosPasFinal = (3,)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
resuelve_problema(problema_6, astar_search, problema_6.h2)

Solución:
	Paso 1: Subir ascensor 1, bajar ascensor 3, subir ascensor (rapido) 5
	Paso 2: Subir ascensor 1, bajar ascensor 3, bajar ascensor (rapido) 5
	Paso 3: Subir pasajero 1 al ascensor 1
	Paso 4: Subir ascensor 1, bajar ascensor 3
	Paso 5: Bajar pasajero 1 del ascensor 1

Tiempo del algoritmo: 27.8053 segundos
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 5.
Nodos analizados: 310


## Problema con 5 ascensores (4 lentos y 1 rápido en pares), 2 personas y 3 bloques 0-4, 4-8 y 8-12:

In [68]:
pisosAsc = (0,4,8,8,0)
pisosPasInicial = (2,4)
pisosPasFinal = (3,11)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 5 ascensores (4 lentos y 1 rápido en pares), 3 personas y 3 bloques 0-4, 4-8 y 8-12:

In [65]:
pisosAsc = (0,4,8,8,0)
pisosPasInicial = (2,4,1)
pisosPasFinal = (3,11,12)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 5 ascensores (4 lentos y 1 rápido en pares), 4 personas y 3 bloques 0-4, 4-8 y 8-12:

In [29]:
pisosAsc = (0,4,8,8,0)
pisosPasInicial = (2,4,1,8)
pisosPasFinal = (3,11,12,1)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.

## Problema con 5 ascensores (4 lentos y 1 rápido en pares), 5 personas y 3 bloques 0-4, 4-8 y 8-12:

In [43]:
pisosAsc = (0,4,8,8,0)
pisosPasInicial = (2,4,1,8,1)
pisosPasFinal = (3,11,12,1,9)
ascVacios = tuple(tuple(0 for x in pisosPasInicial) for x in pisosAsc)
estado_inicial_e6 = (pisosPasInicial,ascVacios,pisosAsc)
objetivo_e6 = (pisosPasFinal,ascVacios,pisosAsc)
# LOS BLOQUES INDICAN LOS PISOS A LOS QUE PUEDEN ACCEDER LOS ASCENSORES
bloques = ((0,4),(4,8),(4,8),(8,12),(0,12))
limAscLento=2
limAscRapido=3
cantAscRapidos=1
problema_6 = ProblemaAscensoresV4(estado_inicial_e6, 
                                  objetivo_e6, 
                                  bloques=bloques, 
                                  limAscLento=limAscLento, 
                                  limAscRapido=limAscRapido, 
                                  cantAscRapidos=cantAscRapidos)
#resuelve_problema(problema_6, astar_search, problema_6.h2)

Al intentar ejecutar el problema inicial para obtener una solución óptima ha tardado mucho tiempo debido a que la heuristica no es admisible.